<a href="https://colab.research.google.com/github/kiranbeethoju/Stock-Market-Forecast/blob/master/stock_predictions_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
ticks_dict = [{"comapny":"AXISBANK.NS","start":"1999-01-01"}, {"comapny":"WIPRO.NS","start":"1996-01-01"},
              {"comapny":"HINDPETRO.NS","start":"1996-01-01"},
              {"comapny":"ONGC.NS","start":"1996-01-01"},
              {"comapny":"INFY.NS","start":"1996-01-01"},
              {"comapny":"TECHM.NS","start":"2006-08-01"},
              {"comapny":"TATACONSUM.NS","start":"2017-06-01"},
              {"comapny":"M&M.NS","start":"1996-01-01",
               "comapny":"TATASTEEL.NS","start":"1996-01-01"}]



In [53]:
!pip install fbprophet
import datetime
import pandas as pd
import numpy as np
import pylab as pl
import datetime
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from matplotlib.collections import LineCollection
from pandas_datareader import data as wb
from sklearn import cluster, covariance, manifold

In [51]:
start = '1996-01-01'
end = '2020-07-15'
#edit your company here
tickers = ['TATASTEEL.NS']

thelen = len(tickers)

price_data = []
for ticker in tickers:
    prices = wb.DataReader(ticker, start = start, end = end, data_source='yahoo')[['Open','Adj Close']]
    price_data.append(prices.assign(ticker=ticker)[['ticker', 'Open', 'Adj Close']])

#names = np.reshape(price_data, (len(price_data), 1))

names = pd.concat(price_data)
names.reset_index()

#pd.set_option('display.max_columns', 500)

open = np.array([q['Open'] for q in price_data]).astype(np.float)
close = np.array([q['Adj Close'] for q in price_data]).astype(np.float)
print(len(names))
names = names.reset_index()
open = names.rename(columns={'Date':'ds','Open':'y'})
open = open[['ds','y']]
close = names.rename(columns={'Date':'ds','Adj Close':'y'})
close = close[["ds","y"]]
from fbprophet import Prophet
from plotly.graph_objs import *

df = open

def run(df,name):
  df = df[6000:]
  #df = df[df['ds'].dt.dayofweek < 5]
  m = Prophet()
  m.fit(df)
  future = m.make_future_dataframe(periods=400)
  future.tail()
  forecast = m.predict(future)
  future = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  future = future.rename(columns={"yhat":"yhat_future","yhat_lower":"yhat_lower_future","yhat_upper":"yhat_upper_future"})
  mixed = pd.merge(df,future,on="ds",how="inner")
  mixed.tail(30)
  avg = (mixed["yhat_future"].iloc[0]+mixed["yhat_lower_future"].iloc[0]+mixed["yhat_upper_future"].iloc[0])/3
  mixed["avg_val"] = ''
  for i in range(0, len(mixed)):
    avg = (mixed["yhat_future"].iloc[i]+mixed["yhat_lower_future"].iloc[i]+mixed["yhat_upper_future"].iloc[i])/3
    mixed['avg_val'].iloc[i] = avg
  mixed = mixed
  import plotly as py

  import plotly.graph_objects as go
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["yhat_future"],
                      mode='lines+markers',
                      name='Predicted Opening Values'))
  fig.show()
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["y"],
                      mode='lines',
                      name='Actual Opening Values'))
  fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["avg_val"],
                      mode='markers',
                      name='Predicted Opening Avg Values '))

  fig.show()
  future.to_csv(f'{name}-{tickers[0]}.csv')
  return mixed
a = run(open,"open")
b = run(close,"close")
a = a.rename(columns={"avg_val":"pred_open_avg","y":"actual_open","yhat_upper_future":"yhat_upper_future_open","yhat_future":"yhat_future_open","yhat_lower_future":"yhat_lower_future_open"})
b = b.rename(columns={"avg_val":"pred_close_avg","y":"actual_close","yhat_upper_future":"yhat_upper_future_close","yhat_future":"yhat_future_close","yhat_lower_future":"yhat_lower_future_close"})
c = pd.merge(a,b,on="ds",how="inner")
df = c[["ds","actual_open","pred_open_avg","actual_close","pred_close_avg"]]
df["open_diff"] = df["pred_open_avg"]-df["actual_open"]
df["close_diff"] = df["pred_close_avg"]-df["actual_close"]
df["actual_raise_of_value"] = df["actual_close"] - df["actual_open"]
df["predicted_raise_of_value"] = df["pred_close_avg"] - df["pred_open_avg"]
df.tail(10)
df.to_csv("{}_complete_analysis.csv".format(tickers[0]))

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


6055


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [54]:
df = pd.read_csv("{}_complete_analysis.csv".format(tickers[0]))
df.head()

,Unnamed: 0,ds,actual_open,pred_open_avg,actual_close,pred_close_avg,open_diff,close_diff,actual_raise_of_value,predicted_raise_of_value
0,0,2020-04-22,266.100006,268.784292,268.350006,269.838396,2.684286,1.488390,2.250000,1.054104
1,1,2020-04-23,270.000000,268.102501,270.299988,272.319229,-1.897499,2.019241,0.299988,4.216728
2,2,2020-04-24,266.299988,267.378680,267.549988,270.110815,1.078692,2.560827,1.250000,2.732135
3,3,2020-04-27,274.000000,272.574017,271.649994,271.508239,-1.425983,-0.141755,-2.350006,-1.065778
4,4,2020-04-28,273.899994,271.401432,273.549988,271.250595,-2.498562,-2.299393,-0.350006,-0.150837


In [55]:
import plotly as py

import plotly.graph_objects as go
fig = go.Figure()
mixed = df
fig.update_layout(
      title="Actual Open vs Actual Close & Pred Open vs Pred Close Prices Trend of =>> {}".format(tickers[0]),
      xaxis_title="Date",
      yaxis_title="Price",
      font=dict(
          family="Courier New, monospace",
          size=16,
          color="#7f7f7f"
      )
  )

fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_open"],
                    mode='lines',
                    name='Actual Opening Values',line=dict(color='mediumspringgreen', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["pred_open_avg"],
                    mode='lines+markers',
                    name='Predicted Opening Values',line=dict(color='mediumseagreen', width=4, dash='dot')))


fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_close"],
                    mode='lines',
                    name='Actual Close Values ',line=dict(color='blue', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["pred_close_avg"],
                    mode='lines',
                    name='Predicted Close Values ',line=dict(color='royalblue', width=4, dash='dot')))
fig.show()
#### Drawing difference between actual vs predicted 


In [56]:
fig.update_layout(
      title="Actual Raise/Fall & Predicted Raise/Fall share value Trend of =>> {}".format(tickers[0]),
      xaxis_title="Date",
      yaxis_title="Price",
      font=dict(
          family="Courier New, monospace",
          size=16,
          color="#7f7f7f"
      )
  )
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["actual_raise_of_value"],
                    mode='lines',
                    name='Actual Opening Values',line=dict(color='yellow', width=4, dash='solid')))
fig.add_trace(go.Scatter(x=mixed["ds"], y=mixed["predicted_raise_of_value"],
                    mode='lines+markers',
                    name='Predicted Opening Values',line=dict(color='red', width=4, dash='dot')))
fig.show()

In [57]:
mdf = pd.read_csv("close-{}.csv".format(tickers[0]))
ndf = pd.read_csv("open-{}.csv".format(tickers[0]))
mdf["Close_Avg_Pred"] = (mdf["yhat_future"]+mdf["yhat_lower_future"]+mdf["yhat_upper_future"])/3
mdf = mdf[["ds","Close_Avg_Pred"]]
ndf["Open_Avg_Pred"] = (ndf["yhat_future"]+ndf["yhat_lower_future"]+ndf["yhat_upper_future"])/3
ndf= ndf[["ds","Open_Avg_Pred"]]
pred_future= pd.merge(mdf,ndf,on="ds",how="inner")
pred_future


,ds,Close_Avg_Pred,Open_Avg_Pred
0,2020-04-22,269.838396,268.784292
1,2020-04-23,272.319229,268.102501
2,2020-04-24,270.110815,267.378680
3,2020-04-27,271.508239,272.574017
4,2020-04-28,271.250595,271.401432
...,...,...,...
450,2021-08-09,595.319329,637.562757
451,2021-08-10,594.265688,638.877556
452,2021-08-11,596.682413,637.176803
453,2021-08-12,599.265986,636.458271


In [58]:
pred_future["expected_raise"] = pred_future["Close_Avg_Pred"]-pred_future["Open_Avg_Pred"]
pred_future.head(60)

,ds,Close_Avg_Pred,Open_Avg_Pred,expected_raise
0,2020-04-22,269.838396,268.784292,1.054104
1,2020-04-23,272.319229,268.102501,4.216728
2,2020-04-24,270.110815,267.378680,2.732135
3,2020-04-27,271.508239,272.574017,-1.065778
4,2020-04-28,271.250595,271.401432,-0.150837
5,2020-04-29,272.821505,272.414918,0.406587
6,2020-04-30,275.139517,271.927166,3.212351
7,2020-05-04,274.302457,276.720649,-2.418192
8,2020-05-05,274.632976,275.950004,-1.317028
9,2020-05-06,275.615441,276.443487,-0.828046
